In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

OUTPUT_DIR = "/home/rohit/DeepRLCacheReplacement/outputs/"

Model = 'ActorCriticQ'

if Model == 'ActorCritic':
    files = ['ActorCritic_50_attention_0.00001.log', 'ActorCritic_50_attention_0.0005.log', 'ActorCritic_50_deep_0.0001.log', 'ActorCritic_50_shallow_0.00005.log', 'ActorCritic_50_attention_0.00005.log', 'ActorCritic_50_deep_0.00001.log', 'ActorCritic_50_deep_0.0005.log', 'ActorCritic_50_shallow_0.0001.log', 'ActorCritic_50_attention_0.0001.log', 'ActorCritic_50_deep_0.00005.log', 'ActorCritic_50_shallow_0.00001.log', 'ActorCritic_50_shallow_0.0005.log']
elif Model == 'Baselines':
    files = ['LRU_50.log', 'MRU_50.log', 'LFU_50.log', 'Random_50.log']
elif Model == "PPO":
    files = ['PPO_50_deep_0.00001_10.log', 'PPO_50_deep_0.00005_20.log', 'PPO_50_deep_0.0001_5.log', 'PPO_50_shallow_0.00005_10.log', 'PPO_50_shallow_0.0001_20.log', 'PPO_50_shallow_0.0005_5.log', 'PPO_50_deep_0.00001_20.log', 'PPO_50_deep_0.00005_5.log', 'PPO_50_shallow_0.00001_10.log', 'PPO_50_shallow_0.00005_20.log', 'PPO_50_shallow_0.0001_5.log', 'PPO_50_shallow_0.001_10.log', 'PPO_50_deep_0.00001_5.log', 'PPO_50_deep_0.0001_10.log', 'PPO_50_shallow_0.00001_20.log', 'PPO_50_shallow_0.00005_5.log', 'PPO_50_shallow_0.0005_10.log', 'PPO_50_shallow_0.001_20.log', 'PPO_50_deep_0.00005_10.log', 'PPO_50_deep_0.0001_20.log', 'PPO_50_shallow_0.00001_5.log', 'PPO_50_shallow_0.0001_10.log', 'PPO_50_shallow_0.0005_20.log', 'PPO_50_shallow_0.001_5.log']
elif Model == "DQN": 
    files = ['DQN_50_0.01.log', 'DQN_50_0.05.log', 'DQN_50_0.1.log', 'DQN_50_0.5.log', 'DQN_50_1.00.log']
elif Model == 'ActorCriticQ':
    files = ['ActorCriticQ_50_attention_0.00001.log', 'ActorCriticQ_50_attention_0.0005.log', 'ActorCriticQ_50_deep_0.0001.log', 'ActorCriticQ_50_shallow_0.00005.log', 'ActorCriticQ_50_attention_0.00005.log', 'ActorCriticQ_50_deep_0.00001.log', 'ActorCriticQ_50_deep_0.0005.log', 'ActorCriticQ_50_shallow_0.0001.log', 'ActorCriticQ_50_attention_0.0001.log', 'ActorCriticQ_50_deep_0.00005.log', 'ActorCriticQ_50_shallow_0.00001.log', 'ActorCriticQ_50_shallow_0.0005.log']

In [ ]:
def parse_logfile(file):
    with open(file, 'r') as f:
        data = f.readlines()
    step_wise = list(filter(lambda x: x.startswith('###'), data))
    info_line = list(filter(lambda x: x.startswith('Namespace'), data))[0]
    info_line = list(map(lambda x: {x.split('=')[0].strip(): x.split('=')[1].strip()}, info_line.replace("Namespace(", '').replace(")", '').replace("'", '').split(',')))

    info_dict = {}
    for d in info_line:
        for k in d:
            info_dict[k] = d[k]
    
    # epoch_wise = list(filter(lambda x: x.startswith('==='), data))
    step_wise = list(map(lambda y: y.replace('###', '').replace(':', '').replace(',','').strip().split(' '), step_wise))
    df = pd.DataFrame(
        map(
            lambda x: {
                'time': float(x[0].replace('Time=', '')),
                'agent': x[1].replace('Agent=',''),
                'cachesize': int(x[2].replace('CacheSize=', '')),
                'episode': int(x[3].replace('Episode=', '')),
                'step': int(x[4].replace('Step=', '')),
                'accesses': int(x[5].replace('Accesses=', '')),
                'misses': int(x[6].replace('Misses=', '')),
                'missrate': float(x[7].replace('MissRate=', '')),
            },
            step_wise
        )
    )
    df = df[['time', 'agent', 'cachesize', 'episode', 'step', 'accesses', 'misses', 'missrate']]
    df['time'] = df['time'].cumsum()

    # df = df[df['episode'] == 0]
    access_in_ep = [0]
    miss_in_ep = [0]
    
    for ep in set(df['episode']):
        last_step_num = df[df['episode'] == ep]['step'].max()
        row = df[(df['episode'] == ep)&(df['step'] == last_step_num)]
        access_in_ep.append(row['accesses'].item())
        miss_in_ep.append(row['misses'].item())
        
        if len(access_in_ep) != 2:
            access_in_ep[-1] += access_in_ep[-2]
            miss_in_ep[-1] += miss_in_ep[-2]

    df['accesses'] = df['accesses'] + np.asarray(list(map(lambda x: access_in_ep[x], df['episode'])))
    df['misses'] = df['misses'] + np.asarray(list(map(lambda x: miss_in_ep[x], df['episode'])))
    df['missrate'] = df['misses'] / df['accesses']
    return info_dict, df

In [ ]:
infos = []
dfs = []
for file in files:
    print(f"[{len(infos)+1}/{len(files)}] Processing {file}")
    op1, op2 = parse_logfile(os.path.join(OUTPUT_DIR, file))
    infos.append(op1)
    dfs.append(op2)

In [ ]:
linestyles = ['-', '--', '-.', ':']

plt.figure(figsize=(12, 5))
if Model == "ActorCritic":
    for i, (a, d) in enumerate(zip(infos, dfs)):
        plt.plot(d['accesses'], d['missrate'], label=f"{a['agent']}_{a['network']}_{a['lr']}", linestyle = linestyles[(i+int(np.ceil(np.random.rand() * 10)))%len(linestyles)])
elif Model == 'Baselines':
    for i, (a, d) in enumerate(zip(infos, dfs)):
        plt.plot(d['accesses'], d['missrate'], label=f"{a['agent']}")
elif Model == 'PPO':
    for i, (a, d) in enumerate(zip(infos, dfs)):
        plt.plot(d['accesses'], d['missrate'], label=f"{a['agent']}_{a['network']}_{a['lr']}_ppoepochs{a['ppo_epochs']}", linestyle = linestyles[(i+int(np.ceil(np.random.rand() * 10)))%len(linestyles)])
elif Model == 'DQN':
    for i, (a, d) in enumerate(zip(infos, dfs)):
        plt.plot(d['accesses'], d['missrate'], label=f"{a['agent']}_{a['lr']}")
elif Model == 'ActorCriticQ':
    for i, (a, d) in enumerate(zip(infos, dfs)):
        plt.plot(d['accesses'], d['missrate'], label=f"{a['agent']}_{a['network']}_{a['lr']}")
    
plt.xlabel("Accesses")
plt.ylabel("MissRate")
plt.legend(loc='upper right', bbox_to_anchor=(1.3, 1))
plt.grid()